In [1]:
# Copyright (c) Microsoft Corporation. All rights reserved
# Licensed under the MIT License.
import fiona
import shapely.geometry
import pandas as pd
import subprocess

In [ ]:
!wget https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_5m.zip
!unzip cb_2018_us_county_5m.zip

: 

In [ ]:
# Read the list of county GEOIDs that make up the Chesapeake Bay
geoids = set(pd.read_csv("../data/chesapeake-bay-county-geoids.csv", dtype={"geoids":"str"})["geoids"].values)

: 

In [ ]:
# Copy the US counties that are in the Chesapeake Bay into a new GeoJSON file
with fiona.open("cb_2018_us_county_5m.shp") as src:
    dst_schema = src.schema.copy()
    dst_schema["geometry"] = "MultiPolygon"
    
    with fiona.open(
        "../data/chesapeake-bay-counties_epsg4269.geojson",
        mode="w",
        driver="GeoJSON",
        crs=src.crs,
        schema=dst_schema
    ) as dst:
        for row in src:
            if row["properties"]["GEOID"] in geoids:
                if row["geometry"]["type"] == "Polygon":
                    shape = shapely.geometry.shape(row["geometry"])
                    geom = shapely.geometry.mapping(
                        shapely.geometry.MultiPolygon(polygons=[shape])
                    )
                    row["geometry"] = geom
                    dst.write(row)
                else:
                    dst.write(row)

: 

In [ ]:
# Convert GeoJSON file to EPSG:4326
assert subprocess.call([
    "ogr2ogr",
    "-t_srs", "epsg:4326",
    "../data/chesapeake-bay-counties_epsg4326.geojson",
    "../data/chesapeake-bay-counties_epsg4269.geojson"
]) == 0

: 

In [ ]:
!rm cb_2018_us_county_5m.*

: 

: 